# Arctic: Arctic - Cloud area fraction
***
### Jupyter open Notebook produced by **[Marine-Analyst.eu](http://marine-analyst.eu)**. Metadata record is available at dataset **[landing page](http://marine-analyst.eu/dev.py?N=simple&O=1253)**.
***
The Marine-Analyst provides augmented data access and reproducible data analysis for marine data. The Marine Analyst is intended for the general public with a focus on educational content. It allows students, teachers and academia to access marine knowledge. Marine topics are developed under the web portal main menu.
The Marine Analyst is an essential tool for oceanographers, scientists, engineers, managers and policy-makers who are analysing the state and dynamics of Europe's seas. The Marine Analyst Jupyter hub provides you with usefull data analysis codes.
Alternatively you can manage your analyses and download links for your data selections via a **[personal dashboard](http://marine-analyst.eu/dev.py?N=simple&O=638&layer=520)**.
This Jupyter notebook is licensed under the **[MIT License](https://github.com/Marine-Analyst/Jupyter/blob/main/LICENSE)**. 

In [ ]:
# Edit the longitude and latitude coordinates to define the geographical area:
###################################
minlon=11.3 #minimum longitude
minlat=53.6 #minimum latitude
maxlon=15.5 #maximum longitude
maxlat=55.9 #maximum latitude
###################################
# This Jupyter Notebook has been automatically generated from R Markdown Notebook and may contain inconsistencies.
# Edit it, execute it and save it as HTML document.
# Anytime you have a question, a suggestion or an issue, you can contact us at my-beach@knowcean.eu.
# Your collaboration is highly appreciated!
###################################
wdpaid=paste(minlon,minlat,maxlon,maxlat,sep="_")
wdpaidsplit <- unlist(strsplit(wdpaid, "[_]"))
xmin <- as.numeric(wdpaidsplit[1])
ymin <- as.numeric(wdpaidsplit[2])
xmax <- as.numeric(wdpaidsplit[3])
ymax <- as.numeric(wdpaidsplit[4])
Sessionid<-"JupyterNotebook1253"

In [ ]:

source_provider <- "Norwegian Meteorological Institute"
source_provider_url <- "https://www.met.no/en/projects/The-weather-model-AROME-Arctic"
layer="cloud_area_fraction"
layer_title<-"AROME ARTIC"
wms_url <- "https://thredds.met.no/thredds/wms/aromearcticlatest/arome_arctic_pp_2_5km_latest.nc?"
layer_id<-1247

In [ ]:

library(rgdal)
library(downloader)
library(httr)
httr::set_config(httr::config(ssl_verifypeer = 0L))
library(ggplot2)
library(mapdata)
library(geojsonio)
library(ggrepel)
library(rasterVis)
library(rgeos)
library(sp)
library(raster)
require(xml2)
library(lubridate)
library(XML)
library(maps)
library(maptools)


# Data information

In [ ]:
# Script for Wekeo environment
sr=SpatialPolygons(list(Polygons(list(Polygon(cbind(c(xmin, xmin, xmax, xmax),c(ymax, ymin, ymin, ymax)))),"1")))
mpa=SpatialPolygonsDataFrame(sr, data.frame(cbind(1:1), row.names=c("1")))
proj4string(mpa)<-CRS("+proj=longlat +datum=WGS84")
bbox<-paste(xmin,ymin,xmax,ymax,sep=",")

In [ ]:


rggbplot <- function(inRGBRst,npix=NA,scale = 'lin'){

rgblinstretch <- function(rgbDf){
maxList <- apply(rgbDf,2,max)
minList <- apply(rgbDf,2,min)
temp<-rgbDf
for(i in c(1:3)){
temp[,i] <- (temp[,i]-minList[i])/(maxList[i]-minList[i])
}
return(temp)
}

rgbeqstretch<-function(rgbDf){

temp<-rgbDf
for(i in c(1:3)){
unique <- na.omit(temp[,i])
if (length(unique>0)){
ecdf<-ecdf(unique)
temp[,i] <- apply(temp[,i,drop=FALSE],2,FUN=function(x) ecdf(x))
}
}
return(temp)
}


npix <- ncell(inRGBRst)

x <- sampleRegular(inRGBRst, size=npix, asRaster = TRUE)
dat <- as.data.frame(x, xy=TRUE)
colnames(dat)[3:5]<-c('r','g','b')

if(scale=='lin'){
dat[,3:5]<- rgblinstretch(dat[,3:5])
} else if(scale=='stretch'){
dat[,3:5]<- rgbeqstretch(dat[,3:5])
}

p <- ggplot()+ geom_tile(data=dat, aes(x=x, y=y, fill=rgb(r,g,b))) + scale_fill_identity()

}


# Geographical extent

## Coordinates

In [ ]:
print (paste("West-Longitude:",round(xmin,2)))
print (paste("South-Latitude:",round(ymin,2)))
print (paste("East-Longitude:",round(xmax,2)))
print (paste("North-Latitude:",round(ymax,2)))

In [ ]:
# Link to Marine Analyst dataset page

link_marineanalyst <- paste0("http://marine-analyst.eu/index.py?maxlat=",ymax,"&maxlon=",xmax,"&minlon=",xmin,"&minlat=",ymin)

## Artic area

In [ ]:
getWMSbasemapXL<-function (xmin,xmax,ymin,ymax)
{
width <- 960
height <- as.integer(width * (ymax-ymin) / (xmax-xmin))

bbox <- paste(xmin, ymin, xmax, ymax, sep = ",")

wms_basemap_url="http://www.gebco.net/data_and_products/gebco_web_services/web_map_service/mapserv"
wms_basemap_layer="gebco_latest"
con<-paste0(wms_basemap_url,"?SERVICE=WMS&VERSION=1.1.0&request=GetMap&layers=",wms_basemap_layer,"&format=image/png&srs=EPSG:4326&bbox=",bbox,"&height=",height,"&width=",width,"")
wms <- "img.png"
wms <- tempfile(wms)
httr::GET(con,write_disk(wms))
basemap <- brick(wms)
names(basemap) <- c("img.1", "img.2", "img.3")

img <- basemap

img@extent@xmin <- xmin
img@extent@ymin <- ymin
img@extent@xmax <- xmax
img@extent@ymax <- ymax
proj4string(img)<-CRS("+proj=longlat +datum=WGS84")
return(img)
}

Xmin <- as.numeric(xmin-1)
Ymin <- as.numeric(ymin-1)
Xmax <- as.numeric(xmax+1)
Ymax <- as.numeric(ymax+1)

wms_imgXL<-getWMSbasemapXL(Xmin,Xmax,Ymin,Ymax)

mapXL <- rggbplot(wms_imgXL)+
geom_polygon(data=mpa,aes(x=long,y=lat,group=group,fill="mpa"),colour="red",fill="blue",alpha=.1)+
coord_quickmap(xlim=range(Xmin,Xmax),ylim=range(Ymin,Ymax))+
ggtitle("Region of interest")+xlab("Longitude")+ylab("Latitude")

plot(mapXL)


In [ ]:


getWMSmap<-function (wms_url,wms_layer,xmin,xmax,ymin,ymax)
{
width <- 960
height <- as.integer(width * (ymax-ymin) / (xmax-xmin))
wms_url<-as.character(wms_url)
wms_layer<-as.character(wms_layer)
bbox <- paste(xmin, ymin, xmax, ymax, sep = ",")

bbox130 <- paste(ymin, xmin, ymax, xmax, sep = ",")
#wms1.3.0 minlat,minlon,maxlat,maxlon
#wms1.3.0 ymin,xmin,ymax,xmax
#con<-paste0(wms_url,"/wms?SERVICE=WMS&VERSION=1.1.0&request=GetMap&layers=",wms_layer,"&format=image/jpeg&srs=EPSG:4326&bbox=",bbox,"&height=",height,"&width=",width,"")
#https://thredds.met.no/thredds/wms/aromearcticlatest/arome_arctic_pp_2_5km_latest.nc?SERVICE=WMS&VERSION=1.3.0&request=GetMap&layers=cloud_area_fraction&format=image/jpeg&crs=EPSG:4326&bbox=76.1,6.4,79.4,31.1&height=128&width=960&styles=
con<-paste0(wms_url,"SERVICE=WMS&VERSION=1.3.0&request=GetMap&layers=",wms_layer,"&format=image/png&styles=&crs=EPSG:4326&bbox=",bbox130,"&height=",height,"&width=",width,"")
wms <- "img.png"
wms <- tempfile(wms)
#download(con, wms, quiet = TRUE, mode = "wb")
httr::GET(con,write_disk(wms))
img <- brick(wms)

img@extent@xmin <- xmin
img@extent@ymin <- ymin
img@extent@xmax <- xmax
img@extent@ymax <- ymax
proj4string(img)<-CRS("+proj=longlat +datum=WGS84")
return(img)
}

getWMSgetFeatureInfo<-function (wms_url,wms_layer,yLat,xLon,xmin,xmax,ymin,ymax)
{
width <- 960
height <- as.integer(width * (ymax-ymin) / (xmax-xmin))
wms_url<-as.character(wms_url)
wms_layer<-as.character(wms_layer)
bbox <- paste(xmin, ymin, xmax, ymax, sep = ",")
#https://thredds.met.no/thredds/wms/aromearcticlatest/arome_arctic_pp_2_5km_latest.nc?LAYERS=wind_speed_of_gust&ELEVATION=10&TIME=2021-03-11T18%3A00%3A00.000Z&TRANSPARENT=true&STYLES=boxfill%2Frainbow&COLORSCALERANGE=0%2C40&NUMCOLORBANDS=20&LOGSCALE=false&SERVICE=WMS&VERSION=1.1.1&REQUEST=GetFeatureInfo&FORMAT=image%2Fpng&SRS=EPSG%3A4326&BBOX=-12.148556%2C39.872978%2C73.324571%2C106.648859&X=223&Y=202&INFO_FORMAT=text%2Fxml&QUERY_LAYERS=wind_speed_of_gust&WIDTH=512&HEIGHT=400&url=https%3A%2F%2Fthredds.met.no%2Fthredds%2Fwms%2Faromearcticlatest%2Farome_arctic_pp_2_5km_latest.nc
con<-paste0(wms_url,"COLORSCALERANGE=0%2C40&NUMCOLORBANDS=20&LOGSCALE=false&SERVICE=WMS&VERSION=1.1.1&STYLES=boxfill/rainbow&request=GetFeatureInfo&query_layers=",wms_layer,"&layers=",wms_layer,"&INFO_FORMAT=text/xml&FEATURE_COUNT=50&X=",xLon,"&Y=",yLat,"&format=image/png&srs=EPSG:4326&TRANSPARENT=true&bbox=",bbox,"&height=",height,"&width=",width,"")
#data <- xmlParse(con)
#xml_data <- xmlToList(data)
xml <- "file.xml"
xml <- tempfile(xml)
httr::GET(con,write_disk(xml))
data <- xml2::read_xml(xml)
xml_data <- xml2::as_list(data)
return(xml_data)
}

# Cloud area fraction

## On the `r toString(Sys.Date())`

In [ ]:



product_title <- "Cloud area fraction"
product_units <- "(%)"
ELEVATION <- 0
layer_title <-"Cloud area fraction"
wms_url <- "https://thredds.met.no/thredds/wms/aromearcticlatest/arome_arctic_pp_2_5km_latest.nc?"
wms_layer <- "cloud_area_fraction"

openlayer<-paste0("http://www.marine-analyst.eu/openlayers3/openlayerxml.py?wms_url=",wms_url,"/wms&wms_layer=",wms_layer,"&bbox=",bbox)

In [ ]:
wms_img<-""
NRT_time_01d0<-''
NRT_mean_01d0<-'no data'

systime<-Sys.Date()
sysh<-format(Sys.time(), "%H")
TIME <- paste0(systime,"T",sysh,"%3A00%3A00.000Z")
wms_url_time <-paste0(wms_url,"TIME=",TIME,"&ELEVATION=",ELEVATION,"&")

tryCatch({wms_img<-getWMSmap(wms_url_time,wms_layer,xmin,xmax,ymin,ymax)
layermeans <- cellStats(wms_img, stat='mean', na.rm=TRUE)
u <- mean(layermeans)

# use row and column numbers:
yLat<-as.integer(dim(wms_img)[1]/2)
xLon<-as.integer(dim(wms_img)[2]/2)
# use spatial dimensions:
y_Lat<-((ymax-ymin)/2+ymin)
x_Lon<-((xmax-xmin)/2+xmin)

}, error=function(e){cat("ERROR: No data available for the defined geographical extent
",conditionMessage(e), "
")})

In [ ]:


tryCatch({
if(is.nan(u) != TRUE) {

# the defined area MUST return a value in its center !!!
xml_data<-getWMSgetFeatureInfo(wms_url_time,wms_layer,yLat,xLon,xmin,xmax,ymin,ymax)

# crop Raster* with Spatial* object
#sub <- as(extent(18.74, 18.76, 77.74, 77.76), 'SpatialPolygons')
sub <- as(extent(x_Lon-res(wms_img)[1], x_Lon+res(wms_img)[1], y_Lat-res(wms_img)[1], y_Lat+res(wms_img)[1]), 'SpatialPolygons')
crs(sub) <- crs(wms_img)
subR <- crop(wms_img, sub)

subRmeans <- cellStats(subR, stat='mean', na.rm=TRUE)
if(is.nan(subRmeans) != TRUE) {
center_mean <- mean(subRmeans)
factorX <- as.numeric(xml_data$FeatureInfoResponse$FeatureInfo$value) / center_mean
nwms_img <- wms_img*factorX
nwms_img_clip <- raster::intersect(nwms_img, mpa)
means3 <- cellStats(nwms_img_clip, stat='mean', na.rm=TRUE)
mean_value_clip <- mean(means3)
#print (paste("Mean value:",mean_value_clip))
#print (paste("Time stamp:",xml_data$FeatureInfoResponse$FeatureInfo$time))

NRT_time_01d0<-substring(xml_data$FeatureInfoResponse$FeatureInfo$time, 1, 13)
NRT_mean_01d0<-sprintf("%.3f",mean_value_clip)

boundaries <- map("worldHires", fill = TRUE, colour = "black", xlim = range(xmin,xmax), ylim = range(ymin,ymax), plot=FALSE)

IDs <- sapply(strsplit(boundaries$names, ":"), function(x) x[1])

coast<- map2SpatialPolygons(boundaries, IDs=IDs, proj4string=CRS("+proj=longlat +datum=WGS84"))

map <- rasterVis::levelplot(nwms_img,  margin = F, contour = F, col.regions = topo.colors(100), main = NRT_time_01d0)+latticeExtra::layer(grid.text(as.numeric(mean_value_clip), 0.5, 0.5), data=list(mean_value_clip=mean_value_clip))
plot(map)

} else {
print("No data available for the defined geographical extent")
}
} else {
print("No data available for the defined geographical extent")
}
}, error=function(e){cat("ERROR: No data available for the defined geographical extent
",conditionMessage(e), "
")})

In [ ]:

print('Mean value')
print(paste("On the: ",as.Date(Sys.Date()),round(as.numeric(mean_value_clip),3),product_units))

mean_value_of_day<-mean_value_clip


In [ ]:




con<-paste0(wms_url,'service=WMS&version=1.3.0&request=GetCapabilities')

xml <- "file.xml"
xml <- tempfile(xml)
httr::GET(con,write_disk(xml))
xmldoc <- xmlParse(xml)

xml_data <- XML::xmlToList(xmldoc)
data.catalog <- data.frame(t(xml_data$Capability$Layer$Layer),row.names=NULL)


for (i in 1:ncol(data.catalog)) {
if (!is.null(data.catalog[[i]]$Layer$Name)) {
if (data.catalog[[i]]$Layer$Name == wms_layer){

print(data.catalog[[i]]$Layer$Name) #list of layers

for (j in 1:length(data.catalog[[i]]$Layer)){

#print(data.catalog[[i]]$Layer[[j]]) #list of parameters
#print(is.atomic(data.catalog[[i]]$Layer[[j]]))
#print(is.null(data.catalog[[i]]$Layer[[j]]$.attrs[["name"]]))

if (is.atomic(data.catalog[[i]]$Layer[[j]]) != TRUE & any(names(data.catalog[[i]]$Layer[[j]]) == ".attrs")){
if (data.catalog[[i]]$Layer[[j]]$.attrs[["name"]] == "time"){
time_values<-as.character(data.catalog[[i]]$Layer[[j]]$text)
}
}

}
}
}
}

#print(time_values)

#2019-01-01T12:00:00.000Z/2021-03-25T12:00:00.000Z/P1D
#P1D means one day increment
list_time_values<-stringr::str_split(time_values, ",", simplify = TRUE)
list_time_values<-trimws(list_time_values, which = c("both", "left", "right"), whitespace = "[ 	
]")

list_YMD_values<- as.Date(list_time_values, "%Y-%m-%dT%H:%M:%S"); list_time_values

for (i in 1:length(list_YMD_values)) {
YMD<-as.Date(list_YMD_values[i])
class(YMD)
}

x<-as.difftime(c(as.Date(max(list_time_values,na.rm=TRUE))-as.Date(min(list_time_values,na.rm=TRUE))), units="hours")
nb_hours<-as.numeric(x, units="hours")

nb_days<-as.integer(Sys.Date()-min(list_YMD_values,na.rm=TRUE))
forecast_days<-as.integer(max(list_YMD_values,na.rm=TRUE)-Sys.Date())



## Forecast

In [ ]:


NRT_time_01d0<-''
NRT_mean_01d0<-NA

MonthLIST <- c()
ValueLIST <- c()
RasterLIST <- c()
RasterLIST<- stack()

for(i in seq(from=0, to=nb_hours, by=4)){

NRT_mean_01d0<-NA

wms_img<-""

systime<-as.POSIXct(Sys.time(), "UTC", "%Y-%m-%dT%H:%M:%S")+60*60*i

sysh<-format(systime, "%H")
systime<-format(as.POSIXct(systime, "UTC", "%Y-%m-%d"),format="%Y-%m-%d")

TIME <- paste0(systime,"T",sysh,"%3A00%3A00.000Z")
record_TIME <- paste0(systime,"T",sysh,":00:00 CET")

wms_url_time <-paste0(wms_url,"TIME=",TIME,"&ELEVATION=",ELEVATION,"&")


tryCatch({wms_img<-getWMSmap(wms_url_time,wms_layer,xmin,xmax,ymin,ymax)
layermeans <- cellStats(wms_img, stat='mean', na.rm=TRUE)
u <- mean(layermeans)

# use row and column numbers:
yLat<-as.integer(dim(wms_img)[1]/2)
xLon<-as.integer(dim(wms_img)[2]/2)
# use spatial dimensions:
y_Lat<-((ymax-ymin)/2+ymin)
x_Lon<-((xmax-xmin)/2+xmin)

}, error=function(e){NRT_mean_01d0 <-NA})


tryCatch({
if(is.nan(u) != TRUE) {

# the defined area MUST return a value in its center !!!
xml_data<-getWMSgetFeatureInfo(wms_url_time,wms_layer,yLat,xLon,xmin,xmax,ymin,ymax)

# crop Raster* with Spatial* object
#sub <- as(extent(18.74, 18.76, 77.74, 77.76), 'SpatialPolygons')
sub <- as(extent(x_Lon-res(wms_img)[1], x_Lon+res(wms_img)[1], y_Lat-res(wms_img)[1], y_Lat+res(wms_img)[1]), 'SpatialPolygons')
crs(sub) <- crs(wms_img)
subR <- crop(wms_img, sub)

subRmeans <- cellStats(subR, stat='mean', na.rm=TRUE)
if(is.nan(subRmeans) != TRUE) {
center_mean <- mean(subRmeans)
factorX <- as.numeric(xml_data$FeatureInfoResponse$FeatureInfo$value) / center_mean
nwms_img <- wms_img*factorX
nwms_img_clip <- raster::intersect(nwms_img, mpa)
means3 <- cellStats(nwms_img_clip, stat='mean', na.rm=TRUE)
mean_value_clip <- mean(means3)

names(nwms_img)<- format(as.POSIXct(format(as.POSIXct(Sys.time(), "UTC", "%Y-%m-%dT%H:%M:%S")+60*60*i, "%Y-%m-%d %H"), format="%Y-%m-%d %H"),format="%Y-%m-%d %H")
RasterLIST<- stack(RasterLIST, nwms_img)

#print (paste("Mean value:",mean_value_clip))
#print (paste("Time stamp:",xml_data$FeatureInfoResponse$FeatureInfo$time))

NRT_time_01d0<-substring(xml_data$FeatureInfoResponse$FeatureInfo$time, 1, 13)
NRT_mean_01d0<-sprintf("%.3f",mean_value_clip)

#map <- rasterVis::levelplot(nwms_img,  margin = F, contour = F, col.regions = topo.colors(100), main = NRT_time_01d2)+latticeExtra::layer(sp.polygons(mpa, col='red', alpha=0.8), data=list(mpa=mpa))+latticeExtra::layer(grid.text(as.numeric(mean_value_clip), 0.5, 0.5), data=list(mean_value_clip=mean_value_clip))
#plot(map)

if(NRT_mean_01d0=="NaN"){NRT_mean_01d0 <-NA}

} else {
NRT_mean_01d0 <-NA
}
} else {
NRT_mean_01d0 <-NA
}
}, error=function(e){NRT_mean_01d0 <-NA})


if(!is.na(NRT_mean_01d0)){
#as.POSIXct(systime, "UTC", "%Y-%m-%dT%H:%M:%S")
#MonthLIST<- c(MonthLIST,format(as.POSIXct(systime, "UTC", "%Y-%m-%dT%H:%M:%S"),format="%Y-%m-%d"))
#MonthLIST<- c(MonthLIST,format(as.POSIXct(systime, "UTC", "%Y-%m-%d"),format="%Y-%m-%d"))
MonthLIST<-c(MonthLIST,format(as.POSIXct(format(as.POSIXct(Sys.time(), "UTC", "%Y-%m-%dT%H:%M:%S")+60*60*i, "%Y-%m-%d %H"), format="%Y-%m-%d %H"),format="%Y-%m-%d %H"))
ValueLIST<- c(ValueLIST, NRT_mean_01d0)
}


}

#MonthLIST <- as.Date(MonthLIST)
ValueLIST <- as.numeric(ValueLIST)
DataLIST<- data.frame(MonthLIST, ValueLIST)
colnames(DataLIST) <- c('DATE','DATA')




In [ ]:



#WINDOWS #```{r, echo=FALSE, animation.hook="gifski", interval = 0.8}
#LINUX #```{r, echo=FALSE, animation.hook='ffmpeg', ffmpeg.format='gif', dev='jpeg'}

#dt<-as.data.frame(MonthLIST)
#RasterLIST <- setZ(RasterLIST, dt[,1], "Datestamp")

makeplot <- function(){
for (i in 1:nlayers(RasterLIST)){
map <- rasterVis::levelplot(RasterLIST[[i]],  margin = F, contour = F, col.regions = topo.colors(100), main = paste0(substr(names(RasterLIST[[i]]), 2, 11),' T',substr(names(RasterLIST[[i]]), 13, 14)))
plot(map)
}
}

#gif_file <- gifski::save_gif(makeplot(), width = 800, height = 450, res = 92)
#utils::browseURL(gif_file)

makeplot()

## Time serie

In [ ]:



map <- ggplot(DataLIST, aes(x = as.POSIXct(DATE, format="%Y-%m-%d %H"), y = DATA, na.rm = TRUE))+ geom_point(color = "#00AFBB", size = 2)+stat_smooth(color = "#FC4E07", fill = "#FC4E07",  method = "loess")+
ggtitle(paste0(Sys.Date(),' to ',max(list_YMD_values,na.rm=TRUE)))+xlab("Time")+ylab(product_units)
plot(map)

